In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import os
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
import csv
import random
import copy

import cv2


# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Define the Hotdog_NotHotdog dataset class
class Hotdog_NotHotdog(torch.utils.data.Dataset):
    def __init__(self, train, transform, data_path='C:/Users/nicla/DTU/ComputerVision/1st Poster/hotdog_nothotdog'):
        'Initialization'
        self.transform = transform
        data_path = os.path.join(data_path, 'train' if train else 'test')
        image_classes = [os.path.split(d)[1] for d in glob.glob(data_path + '/*') if os.path.isdir(d)]
        image_classes.sort()
        self.name_to_label = {c: id for id, c in enumerate(image_classes)}
        self.image_paths = glob.glob(data_path + '/*/*.jpg')
        
        self.images = [self.load_image(path) for path in self.image_paths]
        
    def __len__(self):
        'Returns the total number of samples'
        return len(self.image_paths)

    def load_image(self,path):
        'Generates one sample of data'
        image_path = path
        image = Image.open(image_path)
        c = os.path.split(os.path.split(image_path)[0])[1]
        y = self.name_to_label[c]
        X = self.transform(image)
        return [X, y]

    def __getitem__(self, idx):
        'Generates one sample of data'
        
        return self.images[idx][0], self.images[idx][1]


# Early stopping utility
class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0

    def __call__(self, val_loss):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0


# Function to calculate accuracy
def calculate_accuracy(loader, model, device):
    model.eval()  
    correct = 0
    total = 0
    
    with torch.no_grad(): 
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = (outputs > 0.5).float() 
            correct += (predicted.squeeze() == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    return accuracy

# Training function with early stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience, device):
    model.to(device)  # Move model to the specified device
    train_accuracies = []
    val_accuracies = []
    
    early_stopping = EarlyStopping(patience=patience)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

        for i, data in enumerate(progress_bar):
            inputs, labels = data
            inputs, labels = inputs.to(device,non_blocking=False), labels.to(device, non_blocking=False)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            progress_bar.set_postfix({'Loss': running_loss / (i + 1)})

        # Calculate accuracy on training and validation datasets
        train_acc = calculate_accuracy(train_loader, model, device)
        val_acc = calculate_accuracy(val_loader, model, device)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, '
              f'Train Accuracy: {train_acc:.2f}%, Validation Accuracy: {val_acc:.2f}%')

        # Early stopping check
        early_stopping(val_acc)
        if early_stopping.early_stop:
            print("Early stopping")
            break

    return train_accuracies, val_accuracies

# Main
def run_experiment(augmentation_methods,filename,input_model, num_runs=5,size=128):
    size = size
    batch_size = 64
    num_epochs = 50
    patience = 10
    results = []

    base_transform = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor()
    ])

    # Define augmentation methods
    color_jitter = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
    random_rotation = transforms.RandomRotation(degrees=15)
    random_horizontal_flip = transforms.RandomHorizontalFlip(p=0.5)
    random_affine = transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.8, 1.2))
    random_resized_crop = transforms.RandomResizedCrop(size, scale=(0.8, 1.0))

    augmentation_dict = {
        'color_jitter': color_jitter,
        'random_rotation': random_rotation,
        'random_horizontal_flip': random_horizontal_flip,
        'random_affine': random_affine,
        'random_resized_crop': random_resized_crop
    }

    # Create all combinations of augmentation methods
    augmentation_combinations = [
        [],
        ['color_jitter'],
        ['random_rotation'],
        ['random_horizontal_flip'],
        ['random_affine'],
        ['random_resized_crop'],
        ['color_jitter', 'random_rotation', 'random_horizontal_flip', 'random_affine', 'random_resized_crop']
    ]

    for combo in augmentation_combinations:
        combo_name = '+'.join(combo) if combo else 'no_augmentation'
        print(f"\nRunning experiment with {combo_name}")

        for run in range(num_runs):
            print(f"\nRun {run + 1}/{num_runs}")

            # Create the transform for this combination
            train_transform = transforms.Compose([
                transforms.Resize((size, size)),
                *[augmentation_dict[aug] for aug in combo],
                transforms.ToTensor()
            ])

            # Load and split the datasets
            trainset = Hotdog_NotHotdog(train=True, transform=train_transform, data_path="../hotdog_nothotdog")
            train_size = int(0.8 * len(trainset))
            val_size = len(trainset) - train_size
            train_dataset, val_dataset = random_split(trainset, [train_size, val_size])

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0,pin_memory=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0,pin_memory=True)

            testset = Hotdog_NotHotdog(train=False, transform=base_transform, data_path="../hotdog_nothotdog")
            test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0,pin_memory=True)

            # Initialize the model, loss function, and optimizer
            model = copy.deepcopy(input_model)
            criterion = nn.BCELoss()
            optimizer = optim.Adam(model.parameters(), lr=1e-4)

            # Train the model with early stopping
            train_accuracies, val_accuracies = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience, device)

            # Calculate test accuracy
            test_accuracy = calculate_accuracy(test_loader, model, device)

            # Save results
            results.append({
                'augmentation': combo_name,
                'run': run + 1,
                'train_accuracy': train_accuracies[-1],
                'val_accuracy': val_accuracies[-1],
                'test_accuracy': test_accuracy
            })

    # Save results to CSV
    with open(filename, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['augmentation', 'run', 'train_accuracy', 'val_accuracy', 'test_accuracy'])
        writer.writeheader()
        for result in results:
            writer.writerow(result)

    print("\nExperiment completed. Results saved to ", filename)

In [48]:
# Define the SimpleCNN model
_use_batchnorm = True
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32) if _use_batchnorm else None
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64) if _use_batchnorm else None
        
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(64) if _use_batchnorm else None
        
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 64)
        self.bn_fc1 = nn.BatchNorm1d(64) if _use_batchnorm else None
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.conv1(x)
        if _use_batchnorm:
            x = self.bn1(x)
        x = self.pool(self.relu(x))
        
        x = self.conv2(x)
        if _use_batchnorm:
            x = self.bn2(x)
        x = self.pool(self.relu(x))
        
        x = self.conv3(x)
        if _use_batchnorm:
            x = self.bn3(x)
        x = self.pool(self.relu(x))
        
        x = x.view(-1, 64 * 16 * 16)
        x = self.fc1(x)
        if _use_batchnorm:
            x = self.bn_fc1(x)
        x = self.relu(x)
        
        x = self.fc2(x)
        return torch.sigmoid(x)

class SimpleCNN4(nn.Module):
    def __init__(self):
        super(SimpleCNN4, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32) if _use_batchnorm else None
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64) if _use_batchnorm else None
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(128) if _use_batchnorm else None

        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(128) if _use_batchnorm else None
        
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 16 * 16, 64)
        self.bn_fc1 = nn.BatchNorm1d(64) if _use_batchnorm else None
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.pool(self.relu(x))
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.pool(self.relu(x))
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.pool(self.relu(x))

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.pool(self.relu(x))
        
        x = x.view(-1, 32 * 16 * 16)
        x = self.fc1(x)
        x = self.bn_fc1(x)
        x = self.relu(x)
        
        x = self.fc2(x)
        return torch.sigmoid(x)

class SimpleCNN5(nn.Module):
    def __init__(self):
        super(SimpleCNN5, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32) if _use_batchnorm else None
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64) if _use_batchnorm else None
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(128) if _use_batchnorm else None

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(256) if _use_batchnorm else None

        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(256) if _use_batchnorm else None
        
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 16 * 16, 64)
        self.bn_fc1 = nn.BatchNorm1d(64) if _use_batchnorm else None
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.pool(self.relu(x))
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.pool(self.relu(x))
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.pool(self.relu(x))

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.pool(self.relu(x))

        x = self.conv5(x)
        x = self.bn5(x)
        x = self.pool(self.relu(x))
        
        x = x.view(-1, 16 * 16 * 16)
        x = self.fc1(x)
        x = self.bn_fc1(x)
        x = self.relu(x)
        
        x = self.fc2(x)
        return torch.sigmoid(x)

class SimpleCNN6(nn.Module):
    def __init__(self):
        super(SimpleCNN6, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32) if _use_batchnorm else None
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64) if _use_batchnorm else None
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(128) if _use_batchnorm else None

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(256) if _use_batchnorm else None

        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(512) if _use_batchnorm else None

        self.conv6 = nn.Conv2d(512, 512, kernel_size=3, padding=1, bias=False)
        self.bn6 = nn.BatchNorm2d(512) if _use_batchnorm else None
        
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(8 * 16 * 16, 64)
        self.bn_fc1 = nn.BatchNorm1d(64) if _use_batchnorm else None
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.pool(self.relu(x))
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.pool(self.relu(x))
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.pool(self.relu(x))

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.pool(self.relu(x))

        x = self.conv5(x)
        x = self.bn5(x)
        x = self.pool(self.relu(x))

        x = self.conv6(x)
        x = self.bn6(x)
        x = self.pool(self.relu(x))
        
        x = x.view(-1, 8 * 16 * 16)
        x = self.fc1(x)
        x = self.bn_fc1(x)
        x = self.relu(x)
        
        x = self.fc2(x)
        return torch.sigmoid(x)



class SimpleCNN_custominit(nn.Module):
    def __init__(self):
        super(SimpleCNN_custominit, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32) if _use_batchnorm else None
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64) if _use_batchnorm else None
        
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(64) if _use_batchnorm else None
        
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 64)
        self.bn_fc1 = nn.BatchNorm1d(64) if _use_batchnorm else None
        
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self._initialize_weights()
    def forward(self, x):
        x = self.conv1(x)
        if _use_batchnorm:
            x = self.bn1(x)
        x = self.pool(self.relu(x))
        
        x = self.conv2(x)
        if _use_batchnorm:
            x = self.bn2(x)
        x = self.pool(self.relu(x))
        
        x = self.conv3(x)
        if _use_batchnorm:
            x = self.bn3(x)
        x = self.pool(self.relu(x))
        
        x = x.view(-1, 64 * 16 * 16)
        x = self.fc1(x)
        if _use_batchnorm:
            x = self.bn_fc1(x)
        x = self.relu(x)
        
        x = self.fc2(x)
        return torch.sigmoid(x)
    def _initialize_weights(self):
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.Linear):
                    nn.init.xavier_normal_(m.weight)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)

class SimpleCNN6_custominit(nn.Module):
    def __init__(self):
        super(SimpleCNN6_custominit, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32) if _use_batchnorm else None
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64) if _use_batchnorm else None
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(128) if _use_batchnorm else None

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(256) if _use_batchnorm else None

        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(512) if _use_batchnorm else None

        self.conv6 = nn.Conv2d(512, 512, kernel_size=3, padding=1, bias=False)
        self.bn6 = nn.BatchNorm2d(512) if _use_batchnorm else None
        
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(8 * 16 * 16, 64)
        self.bn_fc1 = nn.BatchNorm1d(64) if _use_batchnorm else None
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)
        self._initialize_weights()
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.pool(self.relu(x))
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.pool(self.relu(x))
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.pool(self.relu(x))

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.pool(self.relu(x))

        x = self.conv5(x)
        x = self.bn5(x)
        x = self.pool(self.relu(x))

        x = self.conv6(x)
        x = self.bn6(x)
        x = self.pool(self.relu(x))
        
        x = x.view(-1, 8 * 16 * 16)
        x = self.fc1(x)
        x = self.bn_fc1(x)
        x = self.relu(x)
        
        x = self.fc2(x)
        return torch.sigmoid(x)

    def _initialize_weights(self):
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.Linear):
                    nn.init.xavier_normal_(m.weight)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)


class SimpleCNN6_big(nn.Module):
    def __init__(self):
        super(SimpleCNN6_big, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32) if _use_batchnorm else None
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64) if _use_batchnorm else None
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(128) if _use_batchnorm else None

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(256) if _use_batchnorm else None

        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(512) if _use_batchnorm else None

        self.conv6 = nn.Conv2d(512, 512, kernel_size=3, padding=1, bias=False)
        self.bn6 = nn.BatchNorm2d(512) if _use_batchnorm else None
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(144*32, 144)
        self.bn_fc1 = nn.BatchNorm1d(144) if _use_batchnorm else None
        
        self.fc2 = nn.Linear(144, 1)
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.pool(self.relu(x))
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.pool(self.relu(x))
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.pool(self.relu(x))

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.pool(self.relu(x))

        x = self.conv5(x)
        x = self.bn5(x)
        x = self.pool(self.relu(x))

        x = self.conv6(x)
        x = self.bn6(x)
        x = self.pool(self.relu(x))
        
        x = x.view(-1, 144*32)
        x = self.fc1(x)
        x = self.bn_fc1(x)
        x = self.relu(x)
        
        x = self.fc2(x)
        return torch.sigmoid(x)

class SimpleCNN6_dropout(nn.Module):
    def __init__(self):
        super(SimpleCNN6_dropout, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32) if _use_batchnorm else None
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64) if _use_batchnorm else None
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(128) if _use_batchnorm else None

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(256) if _use_batchnorm else None

        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(512) if _use_batchnorm else None

        self.conv6 = nn.Conv2d(512, 512, kernel_size=3, padding=1, bias=False)
        self.bn6 = nn.BatchNorm2d(512) if _use_batchnorm else None
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(8 * 16 * 16, 64)
        self.bn_fc1 = nn.BatchNorm1d(64) if _use_batchnorm else None

        self.drop = nn.Dropout2d(0.2)
        self.dropfc = nn.Dropout(0.4)
        
        self.fc2 = nn.Linear(64, 1)
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.pool(self.relu(x))
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.pool(self.relu(x))
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.pool(self.drop(self.relu(x)))

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.pool(self.drop(self.relu(x)))

        x = self.conv5(x)
        x = self.bn5(x)
        x = self.pool(self.drop(self.relu(x)))

        x = self.conv6(x)
        x = self.bn6(x)
        x = self.pool(self.drop(self.relu(x)))
        
        x = x.view(-1, 8 * 16 * 16)
        x = self.fc1(x)
        x = self.bn_fc1(x)
        x = self.relu(x)
        x = self.dropfc(x)
        
        x = self.fc2(x)
        return torch.sigmoid(x)


class SimpleCNN6All(nn.Module):
    def __init__(self):
        super(SimpleCNN6All, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32) if _use_batchnorm else None
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64) if _use_batchnorm else None
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(128) if _use_batchnorm else None

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(256) if _use_batchnorm else None

        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(512) if _use_batchnorm else None

        self.conv6 = nn.Conv2d(512, 512, kernel_size=3, padding=1, bias=False)
        self.bn6 = nn.BatchNorm2d(512) if _use_batchnorm else None
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(144*32, 64)
        self.bn_fc1 = nn.BatchNorm1d(64) if _use_batchnorm else None

        self.drop = nn.Dropout2d(0.2)
        self.dropfc = nn.Dropout(0.4)
        
        self.fc2 = nn.Linear(64, 1)

        self._initialize_weights()
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.pool(self.relu(x))
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.pool(self.relu(x))
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.pool(self.drop(self.relu(x)))

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.pool(self.drop(self.relu(x)))

        x = self.conv5(x)
        x = self.bn5(x)
        x = self.pool(self.drop(self.relu(x)))

        x = self.conv6(x)
        x = self.bn6(x)
        x = self.pool(self.drop(self.relu(x)))
        
        x = x.view(-1, 144*32)
        x = self.fc1(x)
        x = self.bn_fc1(x)
        x = self.relu(x)
        x = self.dropfc(x)
        
        x = self.fc2(x)
        return torch.sigmoid(x)

    def _initialize_weights(self):
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.Linear):
                    nn.init.xavier_normal_(m.weight)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)


In [50]:
# Run the experiment

#model = SimpleCNN()
#run_experiment([],filename="simple.csv",input_model=SimpleCNN(),num_runs=5,size=128)
#run_experiment([],filename="simple4.csv",input_model=SimpleCNN4(),num_runs=5,size=128)
run_experiment([],filename="simple5.csv",input_model=SimpleCNN5(),num_runs=5,size=128)
run_experiment([],filename="simple6.csv",input_model=SimpleCNN6(),num_runs=5,size=128)
#run_experiment([],filename="simple_kaiming.csv",input_model=SimpleCNN_custominit(),num_runs=5,size=128)
run_experiment([],filename="simple6_kaiming.csv",input_model=SimpleCNN6_custominit(),num_runs=5,size=128)
run_experiment([],filename="simple6_big.csv",input_model=SimpleCNN6_big(),num_runs=5,size=224)
run_experiment([],filename="simple6_dropout.csv",input_model=SimpleCNN6_dropout(),num_runs=5,size=128)
run_experiment([],filename="simple6_full.csv",input_model=SimpleCNN6All(),num_runs=5,size=224)


Running experiment with no_augmentation

Run 1/5


Epoch 1/50:  88%|███████████████████████████████████████████████████████████████████████████████████▏          | 23/26 [00:32<00:04,  1.40s/it, Loss=0.551]


KeyboardInterrupt: 

In [ ]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    count = 0
    for name,parameter in model.named_parameters():
        #if not parameter.requires_grad:
           # continue
        params = parameter.numel()
        table.add_row([name, params])
        count += 1
        total_params += params
    print(table)
    print(f"Total Params: {total_params}")
    return total_params

count_parameters(model)